In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from skimage import io
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import cv2
import numpy as np
from plotly.subplots import make_subplots
import os
import os.path as osp

app = JupyterDash(__name__)


def process_list(l):

    if len(l) < num_frames:
        status = f"Bounding box selected for {len(l)} out of {num_frames} images.\n"
    else:
        status = "All bounding boxes have been selected.\n"

    return status + str(l)


@app.callback(
    Output("bbox-data", "children"),
    Input("middle-sai", "relayoutData"),
    prevent_initial_call=True,
)
def on_new_annotation(relayout_data):

    if "shapes" in relayout_data:
        master_dict = {}

        for i in range(len(relayout_data["shapes"])):
            key = relayout_data["shapes"][i]["xref"]
            x0 = int(relayout_data["shapes"][i]["x0"])
            y0 = int(relayout_data["shapes"][i]["y0"])
            x1 = int(relayout_data["shapes"][i]["x1"])
            y1 = int(relayout_data["shapes"][i]["y1"])
            master_dict[key] = [x0, y0, x1, y1]

        l = [master_dict[k] for k in sorted(master_dict.keys())]

        return html.Div([html.Div(process_list(l))], style={"color": "white"})

    else:
        return dash.no_update


class select_bbox:
    def __init__(self, image_dir):

        self.swapped = []

        image_paths = sorted(os.listdir(image_dir))
        self.num_frames = len(image_paths)
        num_rows = 1 if self.num_frames < 5 else 2
        fig = make_subplots(rows=num_rows, cols=4)

        for i in range(num_frames):
            r, c = (i // 4 + 1), (i % 4 + 1)
            img = io.imread(osp.join(image_dir, image_paths[i]))
            fig.add_trace(go.Image(z=img), r, c)

            fig.update_layout(
                autosize=False,
                width=1000,
                height=400,
                dragmode="drawrect",
                newshape=dict(line_color="cyan"),
            )

        config = {
            "modeBarButtonsToAdd": [
                "eraseshape",
            ]
        }

        app.layout = html.Div(
            [
                dcc.Graph(id="middle-sai", figure=fig, config=config),
                html.Pre(id="bbox-data", style={"color": "white"}),
                html.Div(id="all-bboxes", style={"color": "white"}),
            ]
        )

    def show_image(self):
        app.run_server(debug=False, mode="inline")


s = select_bbox("/home/amyxlase/relpose-plus-plus/examples/espresso")
s.show_image()

Dash is running on http://127.0.0.1:8050/



 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [24/Mar/2023 10:06:40] "GET /_alive_81992f87-947b-4074-91f7-d2ddcac07b6e HTTP/1.1" 200 -
